# ➥ Machine Learning Pipelines**

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [2]:
titanic = pd.read_csv('train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# titanic = titanic.iloc[:,[1,2,4,5,6,7,9,11]]
#can be write this way:
titanic.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [4]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Train Datasets

In [5]:
#train dataset
X_train,X_test,y_train,y_test = train_test_split(titanic.drop('Survived',axis='columns'),
                                                    titanic['Survived'],
                                                    test_size=0.2,
                                                    random_state=42)

### Check Null Values

In [6]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

## Imputation Transforming
**If there is any null values in datasets**

In [7]:
tf1 = ColumnTransformer(transformers=[('impute_age',SimpleImputer(),[2]),
                                     ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

In [8]:
tf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [9]:
pd.DataFrame(tf1.fit_transform(X_train))

,0,1,2,3,4,5,6
0,45.5,S,1,male,0,0,28.5
1,23.0,S,2,male,0,0,13.0
2,32.0,S,3,male,0,0,7.925
3,26.0,S,3,male,1,0,7.8542
4,6.0,S,3,female,4,2,31.275
...,...,...,...,...,...,...,...
707,21.0,S,3,female,0,0,7.65
708,29.498846,S,1,male,0,0,31.0
709,41.0,S,3,male,2,0,14.1083
710,14.0,S,1,female,1,2,120.0


## One Hot Encoding

In [10]:
tf2 = ColumnTransformer(transformers=[('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,3])],
                        remainder='passthrough')

In [11]:
pd.DataFrame(tf2.fit_transform(X_train)).shape

(712, 14)

## Scaling

In [12]:
tf3 = ColumnTransformer(transformers=[('scale',MinMaxScaler(),slice(0,14))],
                        remainder='passthrough') #slice(0,14)=apply MinMaxScaler in 0-14 columns

## Feature Extraction
**if needed**

In [13]:
tf4 = SelectKBest(score_func=chi2,k=8)

In [14]:
tf5 = DecisionTreeClassifier()

## Create Pipeline

In [15]:
pipe = Pipeline([('tf1',tf1),
                 ('tf2',tf2),
                 ('tf3',tf3),
                 ('tf4',tf4),
                 ('tf5',tf5)])

In [16]:
pipe.fit(X_train,y_train) #if model is not use then use fit_transform otherwise use fit when model is used.

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('tf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 14, None))])),
                ('tf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002C61D27D080>)),
                ('tf5', DecisionTreeClassifier())])

In [17]:
y_pred = pipe.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy_score(y_test,y_pred)

0.7877094972067039